In [526]:
import yfinance as yf
import numpy as np
import pandas as pd
import pandas_datareader as web
import statsmodels.api as sm

In [529]:
# This version of analytical functions contained an error. Furthermore it is quite chaotic and I have removed a few lines of code
# trying to fix my initial mistake, which means the code may not reflect my process 100%.
#


# Initially the setup was as follows:
#
# 1. pull stock data via fetch_stock_data
# 2. transform stock data to monthly data
# 3. retrieve ff3 data
# 4. use excess_returns function:
#       compute: relative monthly returns
#       compute: monthhly returns - risk free rate (from ff3 dataframe)
# -> here I ran into an error, since Index didnt match between ff3 and stock data
# 
#    first I tried fixing it by converting the timezone data in the ff3 df to DateTimeIndex
#    Then I ran into the problem that one was tz naive and the other aware. 
#

In [528]:
# fetches historical stock data for specified timeframe using yfinance
def fetch_stock_data(ticker_symbol, start_date, end_date):
    data = yf.Ticker(ticker_symbol).history(start = start_date, end = end_date)
    return(data)
    

In [475]:
# resamples a given stock array to specified timeframe, possible inputs: D = Days, H = Hours, ME = Month-End. Since FF3 is monthly
# this may be deleted at a later point
def time_convert(arr, timeframe):
    resample_logic = {"Open" : "first",
                      "High" : "max",
                      "Low" : "min",
                      "Close": "last",
                      "Volume" : "sum",
                     }
    valid_timeframe = ["D","H","ME","W"]

    # Type and Value check
    if not isinstance(timeframe, str):
        raise TypeError(f"Expected str, instead received {type(timeframe)}")
    if timeframe not in valid_timeframe:
        raise ValueError(f"Please enter a valid timeframe. Possible inputs: {valid_timeframe}")

    if timeframe == "D":
        new_arr = arr.resample("D").agg(resample_logic)

    if timeframe == "H":
        new_arr = arr.resample("H").agg(resample_logic)

    if timeframe == "W":
        new_arr = arr.resample("W").agg(resample_logic)

    if timeframe == "ME":
        new_arr = arr.resample("ME").agg(resample_logic)
        
    return new_arr

In [474]:
# fetches per-month FF3 Factor data for specified timeframe using Ken French library
def fetch_ff3_data(start_date, end_date):
    ff_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=start_date, end=end_date)
    del(ff_data["DESCR"], ff_data[1])
    ff_data = pd.DataFrame(ff_data[0]) / 100
    return(ff_data)

In [477]:
apple = fetch_stock_data("AAPL", "2023-12-01", "2024-12-31")
market = fetch_stock_data ("^GSPC", "2023-12-01", "2024-12-31")
ff3 = fetch_ff3_data("2024-01-01", "2024-12-31")

apple_monthly = time_convert(apple, "ME")

/var/folders/lg/8378lyw55rqdlc0qt63jgcy00000gn/T/ipykernel_47995/3360809208.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=start_date, end=end_date)
/var/folders/lg/8378lyw55rqdlc0qt63jgcy00000gn/T/ipykernel_47995/3360809208.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=start_date, end=end_date)


In [479]:
# requires array of stock we want to test to compute excess returns
def excess_returns(arr):
    arr["Excess Returns"] = arr["Close"].pct_change() - ff3["RF"]
    arr = arr.dropna(subset=["Excess Returns"])
    return arr

In [480]:
def regressors():
    X = ff3.iloc[:,0:3]
    sm.add_constant(X.iloc[:,1:4])
    return regressors

In [482]:
TEST

NameError: name 'TEST' is not defined

In [522]:
# fetches monthly relative returns for specified timeframe using yfinance. Can be transformed into stock data loader by removing
# bottom code
def rel_returns(ticker_symbol, start_date, end_date):
    resample_logic = {"Open" : "first",
                      "High" : "max",
                      "Low" : "min",
                      "Close": "last",
                      "Volume" : "sum",
                     }
    data = yf.Ticker(ticker_symbol).history(start = start_date, end = end_date)
    monthly_data = data.resample("ME").agg(resample_logic)
    monthly_returns = monthly_data["Close"].pct_change().dropna()
    monthly_returns.index = ff3.index
    return(monthly_returns)

# initially my code was as follows: 
# 1

In [523]:
apple = rel_returns("AAPL", "2023-12-01", "2024-12-31")

In [524]:
apple

Date
2024-01   -0.042227
2024-02   -0.018543
2024-03   -0.051286
2024-04   -0.006706
2024-05    0.130222
2024-06    0.095553
2024-07    0.054411
2024-08    0.032353
2024-09    0.017467
2024-10   -0.030429
2024-11    0.051707
2024-12    0.062655
Freq: M, Name: Close, dtype: float64